In [1]:
import math

def init_computer(code, inputs):
    return {
        'mem': code.copy(),
        'mem_size': len(code),
        'extend_mem' : {},
        'inst': 0,
        'rel': 0,
        'inputs': inputs.copy(),
        'outputs': [],
        'halt': False,
        'needs_input': False
    }

def read_mem(computer, pos):
    if(pos >= computer['mem_size']):
        if(pos in computer['extend_mem']):
            return computer['extend_mem'][pos]
        else:
            return 0
    else:
        return computer['mem'][pos]

def write_mem(computer, pos, val):
    if(pos < 0):
        print("invalid mem pos %i" % pos)
        return
    if(pos >= computer['mem_size']):
        computer['extend_mem'][pos] = val
    else:
        computer['mem'][pos] = val


def run(computer):
    code_size = len(computer['mem'])
    i = computer['inst']
    outputs = []
    op_info = {1:3, 2:3, 3:1, 4:1, 5:2, 6:2, 7:3, 8:3, 9:1, 99:0}
    computer['needs_input'] = False
    while(True):
        op = read_mem(computer, i)
        opcode = op % 100
        if(not(opcode in op_info)):
            print("error unknown opcode %i" % (opcode))
            computer['needs_input'] = False
            break
        a0 = -1
        a1 = -1
        a2 = -1
        jump = False
        if(op_info[opcode] > 0):
            p_mode = (math.floor(op / 100) % 10)
            if( p_mode == 0 ):
                #position mode (pointer)
                a0 = read_mem(computer, i + 1)
            elif( p_mode == 1 ):
                #immediate mode (value)
                a0 = i + 1
            elif( p_mode == 2 ):
                #relative mode
                a0 = read_mem(computer, i + 1) + computer['rel']
        if(op_info[opcode] > 1):
            p_mode = (math.floor(op / 1000) % 10)
            if( p_mode == 0 ):
                #position mode (pointer)
                a1 = read_mem(computer, i + 2)
            elif( p_mode == 1 ):
                #immediate mode (value)
                a1 = i + 2
            elif( p_mode == 2 ):
                #relative mode
                a1 = read_mem(computer, i + 2) + computer['rel']
        if(op_info[opcode] > 2):
            p_mode = (math.floor(op / 10000) % 10)
            if( p_mode == 0 ):
                #position mode (pointer)
                a2 = read_mem(computer, i + 3)
            elif( p_mode == 1 ):
                #immediate mode (value)
                a2 = i + 3
            elif( p_mode == 2 ):
                #relative mode
                a2 = read_mem(computer, i + 3) + computer['rel']
        if(opcode == 1):
            #add op
            write_mem(computer, a2, read_mem(computer, a0) + read_mem(computer, a1))
        elif(opcode == 2):
            #mult op
            write_mem(computer, a2, read_mem(computer, a0) * read_mem(computer, a1))
        elif(opcode == 3):
            #read op
            if(len(computer['inputs']) == 0):
                computer['needs_input'] = True
                break
            write_mem(computer, a0, computer['inputs'][0])
            computer['inputs'] = computer['inputs'][1:]
        elif(opcode == 4):
            outputs.append(read_mem(computer, a0))
        elif(opcode == 5):
            #jump if true op
            if(read_mem(computer, a0) != 0):
                jump = True
                i = read_mem(computer, a1)
        elif(opcode == 6):
            #jump if false op
            if(read_mem(computer, a0) == 0):
                jump = True
                i = read_mem(computer, a1)
        elif(opcode == 7):
            #check less than op
            write_mem(computer, a2, 1 if(read_mem(computer, a0) < read_mem(computer, a1)) else 0)
        elif(opcode == 8):
            #check equals op
            write_mem(computer, a2, 1 if(read_mem(computer, a0) == read_mem(computer, a1)) else 0)
        elif(opcode == 9):
            #change relative param op
            computer['rel'] = computer['rel'] + read_mem(computer, a0)
        elif(opcode == 99):
            #halt op
            computer['halt'] = True
            computer['needs_input'] = False
            break
        if(not(jump)):
            i = i + op_info[opcode] + 1
        if(i >= code_size):
            print('exiting b/c end of code reached')
            computer['needs_input'] = False
    computer['outputs'] = outputs
    computer['inst'] = i
    
    return computer


In [8]:
import random

class vec2i(object):
    def __init__(self, x, y):
        self.x = int(round(x))
        self.y = int(round(y))
    
    #@classmethod
    #def from_vec2f(cls, vf):
        #return cls(int(round(vf.x)), int(round(vf.y)))

    @classmethod
    def from_index(cls, ind):
        ssplit = ind[1:].split('P')
        yoff = 1
        if(len(ssplit) == 1):
            ssplit = ind[1:].split('N')
            yoff = -1
        
        xoff = 1
        if(ind[0] == 'N'):
            xoff = -1
        
        return cls(xoff * int(ssplit[0]), yoff * int(ssplit[1]))
        
    def index(self):
        xpart = ""
        if(self.x < 0):
            xpart = "N%i" % abs(self.x) 
        else:
            xpart = "P%i" % abs(self.x)
        ypart = ""
        if(self.y < 0):
            ypart = "N%i" % abs(self.y) 
        else:
            ypart = "P%i" % abs(self.y)
        return "%s%s" % (xpart,ypart)
    
    def step(self, v2):
        return vec2i(self.x + v2.x, self.y + v2.y)
    
    def diff(self, v2):
        return vec2i(v2.x - self.x, v2.y - self.y)
    
    def copy(self):
        return vec2i(self.x, self.y)
    
    def __str__(self):
        return ("(%i,%i)" % (self.x, self.y))
    
    def __repr__(self):
        return ("(%i,%i)" % (self.x, self.y))
    
class Area(object):
    def __init__(self):
        self.map = {}
        self.leftmost = 0
        self.upmost = 0
        self.rightmost = 0
        self.downmost = 0
        self.move_steps = {1:vec2i(0,-1), 2:vec2i(0,1), 3:vec2i(-1,0), 4:vec2i(1,0)}
        self.parent = {}
        self.parent_move = {}
        self.open = set()
        self.closed = set()
        self.mapscore = {}
        self.finaldest = ''
        self.floodval = {}
        self.floodopen = set()
        self.floodclosed = set()
        
        startind = vec2i(0,0).index()
        self.closed.add(startind)
        self.parent[startind] = 'ROOT'
        self.mapscore[startind] = 0
        for idir in range(1,5):
            dirind = self.move_steps[idir].index()
            self.open.add(dirind)
            self.parent[dirind] = startind
            self.mapscore[dirind] = 1
            self.parent_move[dirind] = self.rev_dir(idir)
        
    def rev_dir(self, forward):
        if(forward == 1):
            return 2
        elif(forward == 2):
            return 1
        elif(forward == 3):
            return 4
        elif(forward == 4):
            return 3
        
    def wall(self, v):
        key = v.index()
        if(not(key in self.map)):
            return 0
        return self.map[key]
    
    def wallind(self, key):
        if(not(key in self.map)):
            return 0
        return self.map[key]
    
    def path_to_root(self, v):
        cursor = v
        path = [cursor]
        while(cursor != 'ROOT'):
            if(not(cursor in self.parent)):
                print('couldn\'t find parent for %s' % cursor)
                return None
            cursor = self.parent[cursor]
            path.append(cursor)
        return path
    
    def moves_between(self, v1, v2):
        path1 = self.path_to_root(v1)
        path2 = self.path_to_root(v2)
        
        smaller = len(path1)
        if(len(path2) < smaller):
            smaller = len(path2)
        i = -1
        while(True):
            if(i < -smaller):
                break
            if(path1[i] != path2[i]):
                break
            i = i -1
        path2_part = path2[:i+1]
        path2_part.reverse()
        return [self.parent_move[p1] for p1 in path1[:i+1]] + [self.rev_dir(self.parent_move[p2]) for p2 in path2_part]
    
    def set_map(self, v, kind):
        key = v.index()
        
        if(not(key in self.closed)):
            self.map[key] = kind
            if(kind == 3):
                self.finaldest = key
            if(key in self.open):
                self.open.remove(key)
                self.closed.add(key)
            if(v.x < self.leftmost):
                self.leftmost = v.x
            if(v.y < self.upmost):
                self.upmost = v.y
            if(v.x > self.rightmost):
                self.rightmost = v.x
            if(v.y > self.downmost):
                self.downmost = v.y

    def move(self, v1, v2, kind):
        from_key = v1.index()
        to_key = v2.index()
        
        for idir in range(1,5):
            s = v2.step(self.move_steps[idir]).index()
            if(not(s in self.closed) and not(s in self.open)):
                self.open.add(s)
                self.parent[s] = to_key
                self.mapscore[s] = self.mapscore[to_key] + 1
                self.parent_move[s] = self.rev_dir(idir)
            
        if(not(to_key in self.closed)):
            self.set_map(v2, kind)
            
    def render(self, v = None):
        width = self.rightmost - self.leftmost + 1
        height = self.downmost - self.upmost + 1
        image = ''
        for y in range(height):
            for x in range(width):
                cursor = vec2i(x + self.leftmost, y + self.upmost)
                mapind = self.wall(cursor) % 10
                if(v!= None and cursor.x == v.x and cursor.y == v.y):
                    image = image + 'R'
                elif(mapind == 0): #unknown
                    image = image + ' '
                elif(mapind == 1): #floor
                    image = image + '.'
                elif(mapind == 2): #wall
                    image = image + '#'
                elif(mapind == 3): #oxygen_system
                    image = image + 'O'
            image = image + '\n'
        return image
    
    def flood(self):
        if(self.finaldest == ''):
            print("can't flood, haven't found final dest!")
            return
        
        self.floodval[self.finaldest] = 0
        self.floodclosed.add(self.finaldest)
        
        v = vec2i.from_index(self.finaldest)
        openlist = []
        opencount = 0
        for i in range(1, 5):
            s = v.step(self.move_steps[i]).index()
            kind = self.wallind(s)
            if(kind == 1): #floor
                self.floodopen.add(s)
                openlist.append(s)
                opencount = opencount + 1
                self.floodval[s] = 1
        
        highestspilltime = 0
        
        while(opencount > 0):
            nexts = openlist[0]
            openlist = openlist[1:]
            opencount = opencount - 1
            self.floodopen.remove(nexts)
            self.floodclosed.add(nexts)
            v = vec2i.from_index(nexts)
            spilltime = self.floodval[nexts] + 1
            
            for i in range(1, 5):
                s = v.step(self.move_steps[i]).index()
                if((s in self.floodclosed) or (s in self.floodopen)):
                    continue
                    
                kind = self.wallind(s)
                if(kind == 1): #floor
                    self.floodopen.add(s)
                    openlist.append(s)
                    opencount = opencount + 1
                    self.floodval[s] = spilltime
                    if(spilltime > highestspilltime):
                        highestspilltime = spilltime
        
        return highestspilltime
            
            
                

class Droid(object):
    def __init__(self, code, area):
        self.computer = init_computer(code, [])
        self.area = area
        self.pos = vec2i(0,0)
        self.move_steps = {1:vec2i(0,-1), 2:vec2i(0,1), 3:vec2i(-1,0), 4:vec2i(1,0)}
        self.dir_priority = [1,3,2,4]
        self.randstrats = {0:[1,1,1,1,1,1,1,1,1,1],
                           1:[2,2,2,2,2,2,2,2,2,2,2],
                           2:[3,3,3,3,3,3,3,3],
                           3:[4,4,4,4,4,4,4,4,4],
                           4:[1,2,2,3,3,3,4,4,4,4,1,1,1,1,1,2,2,2,2,2,2,3,3,3,3,3,3,3,4,4,4,4,4,4,4,4]}
        
    def exec1(self):
        inputs = self.computer['inputs'].copy()
        self.computer = run(self.computer)
        outputs = self.computer['outputs']
        self.computer['outputs'] = []
        
        if(len(inputs) != len(outputs)):
            print('different number of inputs(%i) and outputs(%i)' % (len(inputs), len(outputs)))
            return
        
        for i in range(len(inputs)):
            stepin = inputs[i]
            stepout = outputs[i]
            
            if(stepout == 0): #hit wall, didn't move
                self.area.set_map(self.pos.step(self.move_steps[stepin]),2)
            elif(stepout == 1): #moved onto floor
                pos_to = self.pos.step(self.move_steps[stepin])
                self.area.move(self.pos, pos_to, 1)
                self.pos = pos_to
            elif(stepout == 2): #moved into oxygen system
                pos_to = self.pos.step(self.move_steps[stepin])
                self.area.move(self.pos, pos_to, 3)
                self.pos = pos_to
            else:
                print('unknown output %i' % stepout)
    
    def execmult(self, ins):
        self.computer['inputs'] = ins
        self.exec1()
        
    def execrand(self):
        strat = random.randint(0, 9)
        if(strat < 5):
            self.execmult(self.randstrats[strat].copy())
        else:
            numsteps = random.randint(0,12)
            steps = []
            for i in range(numsteps):
                steps.append(random.randint(1,4))
            self.execmult(steps)
            
    def autopilot(self):
        if(self.computer['halt']):
            print("can't autopilot, computer has shut down")
            return
        opens = list(self.area.open)
        if(len(opens) == 0):
            print('the world is smaller than it used to be')
        else:
            dest = opens[random.randint(0,len(opens)-1)]
            moves = self.area.moves_between(self.pos.index(), dest)
            #print('from %s => %s' % (self.pos.index(), dest))
            #print("moves: ", end ="")
            #print(moves)
            self.execmult(moves)
    
    def render(self):
        print(self.area.render(self.pos))
        print(self.area.render())

class Scaffolding(object):
    
    def __init__(self, initstr):
        self.map = []
        row = 0
        col = 0
        self.robotpos = None
        self.width = 0
        self.height = 0
        for c in initstr:
            if(col == 0):
                self.map.append([])
            if(c == '.'):
                self.map[row].append(0)
            elif(c == '\n'):
                row = row + 1
                self.width = col
                col = -1
            else:
                self.map[row].append(1)
                if(c != '#'):
                    self.robotpos = vec2i(col, row)
            col = col + 1
        self.height = row
            
        
    def is_scaffolding(self, x, y):
        if(x < 0 or x >= self.width or y < 0 or y >= self.height):
            return False
        return self.map[y][x] == 1
    
    def is_intersection(self, x, y):
        if(not(self.is_scaffolding(x,y))):
            return False
        if(not(self.is_scaffolding(x-1,y))):
            return False
        if(not(self.is_scaffolding(x+1,y))):
            return False
        if(not(self.is_scaffolding(x,y-1))):
            return False
        if(not(self.is_scaffolding(x,y+1))):
            return False
        return True
    
    def calc_align_params(self):
        params = 0
        
        for y in range(self.height):
            for x in range(self.width):
                if(self.is_intersection(x,y)):
                    params = params + x * y
        return params
        

In [9]:
#area = Area()
code = [1,330,331,332,109,3016,1101,1182,0,16,1101,1441,0,24,102,1,0,570,1006,570,36,1002,571,1,0,1001,570,-1,570,1001,24,1,24,1106,0,18,1008,571,0,571,1001,16,1,16,1008,16,1441,570,1006,570,14,21101,58,0,0,1105,1,786,1006,332,62,99,21101,333,0,1,21101,73,0,0,1105,1,579,1101,0,0,572,1101,0,0,573,3,574,101,1,573,573,1007,574,65,570,1005,570,151,107,67,574,570,1005,570,151,1001,574,-64,574,1002,574,-1,574,1001,572,1,572,1007,572,11,570,1006,570,165,101,1182,572,127,102,1,574,0,3,574,101,1,573,573,1008,574,10,570,1005,570,189,1008,574,44,570,1006,570,158,1106,0,81,21101,340,0,1,1106,0,177,21102,1,477,1,1106,0,177,21101,514,0,1,21102,176,1,0,1105,1,579,99,21102,1,184,0,1105,1,579,4,574,104,10,99,1007,573,22,570,1006,570,165,1002,572,1,1182,21101,0,375,1,21101,211,0,0,1105,1,579,21101,1182,11,1,21101,222,0,0,1105,1,979,21102,1,388,1,21102,1,233,0,1105,1,579,21101,1182,22,1,21102,244,1,0,1106,0,979,21102,401,1,1,21101,0,255,0,1106,0,579,21101,1182,33,1,21101,266,0,0,1106,0,979,21102,414,1,1,21101,0,277,0,1105,1,579,3,575,1008,575,89,570,1008,575,121,575,1,575,570,575,3,574,1008,574,10,570,1006,570,291,104,10,21102,1,1182,1,21101,0,313,0,1106,0,622,1005,575,327,1102,1,1,575,21101,327,0,0,1106,0,786,4,438,99,0,1,1,6,77,97,105,110,58,10,33,10,69,120,112,101,99,116,101,100,32,102,117,110,99,116,105,111,110,32,110,97,109,101,32,98,117,116,32,103,111,116,58,32,0,12,70,117,110,99,116,105,111,110,32,65,58,10,12,70,117,110,99,116,105,111,110,32,66,58,10,12,70,117,110,99,116,105,111,110,32,67,58,10,23,67,111,110,116,105,110,117,111,117,115,32,118,105,100,101,111,32,102,101,101,100,63,10,0,37,10,69,120,112,101,99,116,101,100,32,82,44,32,76,44,32,111,114,32,100,105,115,116,97,110,99,101,32,98,117,116,32,103,111,116,58,32,36,10,69,120,112,101,99,116,101,100,32,99,111,109,109,97,32,111,114,32,110,101,119,108,105,110,101,32,98,117,116,32,103,111,116,58,32,43,10,68,101,102,105,110,105,116,105,111,110,115,32,109,97,121,32,98,101,32,97,116,32,109,111,115,116,32,50,48,32,99,104,97,114,97,99,116,101,114,115,33,10,94,62,118,60,0,1,0,-1,-1,0,1,0,0,0,0,0,0,1,14,0,0,109,4,1202,-3,1,587,20102,1,0,-1,22101,1,-3,-3,21102,0,1,-2,2208,-2,-1,570,1005,570,617,2201,-3,-2,609,4,0,21201,-2,1,-2,1105,1,597,109,-4,2105,1,0,109,5,1202,-4,1,630,20101,0,0,-2,22101,1,-4,-4,21102,0,1,-3,2208,-3,-2,570,1005,570,781,2201,-4,-3,652,21002,0,1,-1,1208,-1,-4,570,1005,570,709,1208,-1,-5,570,1005,570,734,1207,-1,0,570,1005,570,759,1206,-1,774,1001,578,562,684,1,0,576,576,1001,578,566,692,1,0,577,577,21101,0,702,0,1106,0,786,21201,-1,-1,-1,1105,1,676,1001,578,1,578,1008,578,4,570,1006,570,724,1001,578,-4,578,21101,0,731,0,1105,1,786,1105,1,774,1001,578,-1,578,1008,578,-1,570,1006,570,749,1001,578,4,578,21101,756,0,0,1106,0,786,1105,1,774,21202,-1,-11,1,22101,1182,1,1,21101,774,0,0,1106,0,622,21201,-3,1,-3,1105,1,640,109,-5,2106,0,0,109,7,1005,575,802,20101,0,576,-6,20101,0,577,-5,1106,0,814,21101,0,0,-1,21101,0,0,-5,21102,1,0,-6,20208,-6,576,-2,208,-5,577,570,22002,570,-2,-2,21202,-5,45,-3,22201,-6,-3,-3,22101,1441,-3,-3,2101,0,-3,843,1005,0,863,21202,-2,42,-4,22101,46,-4,-4,1206,-2,924,21102,1,1,-1,1105,1,924,1205,-2,873,21101,35,0,-4,1106,0,924,1201,-3,0,878,1008,0,1,570,1006,570,916,1001,374,1,374,1202,-3,1,895,1101,2,0,0,1201,-3,0,902,1001,438,0,438,2202,-6,-5,570,1,570,374,570,1,570,438,438,1001,578,558,922,20101,0,0,-4,1006,575,959,204,-4,22101,1,-6,-6,1208,-6,45,570,1006,570,814,104,10,22101,1,-5,-5,1208,-5,35,570,1006,570,810,104,10,1206,-1,974,99,1206,-1,974,1101,0,1,575,21101,973,0,0,1105,1,786,99,109,-7,2106,0,0,109,6,21102,0,1,-4,21102,1,0,-3,203,-2,22101,1,-3,-3,21208,-2,82,-1,1205,-1,1030,21208,-2,76,-1,1205,-1,1037,21207,-2,48,-1,1205,-1,1124,22107,57,-2,-1,1205,-1,1124,21201,-2,-48,-2,1105,1,1041,21102,-4,1,-2,1105,1,1041,21101,-5,0,-2,21201,-4,1,-4,21207,-4,11,-1,1206,-1,1138,2201,-5,-4,1059,1202,-2,1,0,203,-2,22101,1,-3,-3,21207,-2,48,-1,1205,-1,1107,22107,57,-2,-1,1205,-1,1107,21201,-2,-48,-2,2201,-5,-4,1090,20102,10,0,-1,22201,-2,-1,-2,2201,-5,-4,1103,2101,0,-2,0,1106,0,1060,21208,-2,10,-1,1205,-1,1162,21208,-2,44,-1,1206,-1,1131,1106,0,989,21102,439,1,1,1105,1,1150,21102,477,1,1,1106,0,1150,21101,514,0,1,21102,1149,1,0,1105,1,579,99,21101,1157,0,0,1105,1,579,204,-2,104,10,99,21207,-3,22,-1,1206,-1,1138,2101,0,-5,1176,2101,0,-4,0,109,-6,2106,0,0,10,5,40,1,44,1,44,1,44,7,44,1,44,1,7,13,24,1,7,1,11,1,8,7,9,1,7,1,11,1,8,1,5,1,9,1,7,1,11,1,8,1,5,1,9,1,1,5,1,1,5,9,6,1,5,1,9,1,1,1,3,1,1,1,5,1,5,1,1,1,6,1,5,1,7,11,1,11,1,1,6,1,5,1,7,1,1,1,1,1,3,1,3,1,3,1,7,1,6,1,5,1,7,1,1,7,3,1,3,1,7,1,6,1,5,1,7,1,3,1,7,1,3,1,7,1,6,1,5,1,1,11,1,11,7,1,6,1,5,1,1,1,5,1,5,1,5,1,11,1,6,1,5,9,5,1,5,1,11,1,6,1,7,1,11,1,5,1,11,1,6,7,1,1,11,1,5,7,5,7,6,1,1,1,11,1,11,1,11,1,6,1,1,13,11,1,11,1,6,1,25,1,11,1,6,1,25,1,11,1,6,1,25,1,11,1,6,1,25,1,11,1,6,1,25,1,11,1,6,1,25,1,1,11,6,1,25,1,1,1,16,7,19,7,40,1,3,1,40,1,3,1,40,1,3,1,40,5,6]
#droid = Droid(code, area)
#droid.exec1()
computer = init_computer(code, [])
run(computer)
mapstr = ''.join(chr(i) for i in computer['outputs'][:-1])

scaffolding = Scaffolding(mapstr)
print('part 1: align params = %i' % scaffolding.calc_align_params())

part 1: align params = 3448


In [10]:
print(mapstr)

..........####^..............................
..........#..................................
..........#..................................
..........#..................................
..........#######............................
................#............................
................#.......#############........
................#.......#...........#........
#######.........#.......#...........#........
#.....#.........#.......#...........#........
#.....#.........#.#####.#.....#########......
#.....#.........#.#...#.#.....#.....#.#......
#.....#.......###########.###########.#......
#.....#.......#.#.#...#...#...#.......#......
#.....#.......#.#######...#...#.......#......
#.....#.......#...#.......#...#.......#......
#.....#.###########.###########.......#......
#.....#.#.....#.....#.....#...........#......
#.....#########.....#.....#...........#......
#.......#...........#.....#...........#......
#######.#...........#.....#######.....#######
......#.#...........#...........#.

In [15]:
routine = [ord(c) for c in 'A,A,B,C,C,A,C,B,C,B\n']
funcA = [ord(c) for c in 'L,4,L,4,L,6,R,10,L,6\n']
funcB = [ord(c) for c in 'L,12,L,6,R,10,L,6\n']
funcC = [ord(c) for c in 'R,8,R,10,L,6\n']
livefeed = [ord(c) for c in 'n\n']
code2 = code.copy()
code2[0] = 2
computer_p2 = init_computer(code2, routine + funcA + funcB + funcC + livefeed )
computer_p2 = run(computer_p2)
print('part 2: dust = %i' % computer_p2['outputs'][-1])

part 2: dust = 762405
